In [1]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [2]:
try:
    # print(dir(fablib))
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MASS         4  125/128  1016/1024  55750/55800    253/254                 2/2                             0/0                            6/6                 0/0               3/3
MAX         10  258/320  2366/2560  115348/116400  595/635                 1/2                             3/4                            16/16               4/4               6/6
TACC        10  274/320  2388/2560  116102/116400  605/635                 0/2                             1/4                            16/16               4/4               6/6
MICH         6  192/192  1536/1536  60600/60600    381/381                 2/2

In [3]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Slice 
slice_name = 'playground2'

# [site1,site2,site3] = fablib.get_random_sites(count=3)
site1 = site2 = site3 = 'MAX'
print(f"Sites: {site1},{site2},{site3}")


# Switches
s1_name = "s1"

switch_cores = 2
switch_ram = 8
switch_disk = 100

# Hosts
h1_name = "h1"
h2_name = "h2"

h1_subnet=IPv4Network('10.0.1.0/24')
h1_addr=IPv4Address('10.0.1.1')

h2_subnet=IPv4Network('10.0.2.0/24')
h2_addr=IPv4Address('10.0.2.2')

host_cores = 2
host_ram = 8
host_disk = 10

net_h1_name = 'net_h1'
net_h2_name = 'net_h2'


# All node properties
image = 'default_ubuntu_20'

Sites: MAX,MAX,MAX


In [4]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node s1
    s1 = slice.add_node(name=s1_name, site=site1,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    print(dir(s1))
    s1_iface_local = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()[0]
    s1_iface_local2 = s1.add_component(model='NIC_Basic', name="s1_local_nic2").get_interfaces()[0]
   
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_iface_local])
    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s1_iface_local2])
    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


-----------  ------------------------------------
Slice Name   playground2
Slice ID     b395a0fd-1c62-411c-a93b-17ea083c92c9
Slice State  StableOK
Lease End    2022-09-02 22:27:49 +0000
-----------  ------------------------------------

Retry: 13, Time: 149 sec

ID                                    Name    Site    Host                         Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  ------  ------  -------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
477844de-2b92-4d01-b183-3821ac0214c2  s1      MAX     max-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  63.239.135.73    Active

Time to stable 149 seconds
Running post_boot_config ... Time to post boot config 154 seconds

Name                 Node    Network      Bandwidth  VLAN    MAC                Physical OS Interface    OS Interface
-------------------  ------  ---------  -----------  ------  

In [5]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        # print(f"   Cores             : {node.get_cores()}")
        # print(f"   RAM               : {node.get_ram()}")
        # print(f"   Disk              : {node.get_disk()}")
        # print(f"   Image             : {node.get_image()}")
        # print(f"   Image Type        : {node.get_image_type()}")
        # print(f"   Host              : {node.get_host()}")
        # print(f"   Site              : {node.get_site()}")
        # print(f"   Management IP     : {node.get_management_ip()}")
        # print(f"   Reservation ID    : {node.get_reservation_id()}")
        # print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        # print(f"   Components        :  ")
        # for component in node.get_components():
        #     print(f"      Name             : {component.get_name()}")
        #     print(f"      Details          : {component.get_details()}")
        #     print(f"      Disk (G)         : {component.get_disk()}")
        #     print(f"      Units            : {component.get_unit()}")
        #     print(f"      PCI Address      : {component.get_pci_addr()}")
        #     print(f"      Model            : {component.get_model()}")
        #     print(f"      Type             : {component.get_type()}") 
        # print(f"   Interfaces        :  ")
        for interface in node.get_interfaces():
        #     print(f"       Name                : {interface.get_name()}")
        #     print(f"           Bandwidth           : {interface.get_bandwidth()}")
            # print(f"           VLAN                : {interface.get_vlan()}")  
            # print(f"           MAC                 : {interface.get_mac()}") 
            print(f"           OS iface name       : {interface.get_os_interface()}")
    # for network in slice.get_l2networks():
    #     print("Network:")
    #     print(f"    Name:            {network.get_name()}")
    # print(f"Interface Map: {slice.get_interface_map()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : s1
   SSH Command       : ssh -i /home/fabric/work/fabric_config/.ssh/id_rsa -J durbek_gafurov_0000000854@bastion-1.fabric-testbed.net ubuntu@63.239.135.73
           OS iface name       : ens8
           OS iface name       : ens7


In [6]:
def checkInetConnection(node):
    try:

        #If the node is an IPv6 Node then configure NAT64
        if type(ip_address(node.get_management_ip())) is IPv6Address:
            node.upload_file('nat64.sh','nat64.sh')

            stdout, stderr = node.execute(f'chmod +x nat64.sh && ./nat64.sh')
            print(stdout)
            print(stderr)
        else:
            print("not ipv6")

        #Access non-IPv6 Services
        stdout, stderr = node.execute(f'sudo apt-get update -qq && sudo apt install net-tools && git clone https://github.com/fabric-testbed/jupyter-examples.git')
        # print(stdout)
        print(stderr)

        stdout, stderr = node.execute(f'ls jupyter-examples')
        print(stdout)
        print(stderr)
        
        stdout, stderr = node.execute(f'ifconfig')
        print(stdout)
        print(stderr)


    except Exception as e:
        print(f"Exception: {e}")

In [7]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        checkInetConnection(node)
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : s1
not ipv6


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Cloning into 'jupyter-examples'...

LICENSE
Readme.md
fabric_examples
start_here.ipynb


ens3: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000
        inet 10.20.5.97  netmask 255.255.254.0  broadcast 10.20.5.255
        inet6 fe80::f816:3eff:fec8:15f1  prefixlen 64  scopeid 0x20<link>
        ether fa:16:3e:c8:15:f1  txqueuelen 1000  (Ethernet)
        RX packets 4354  bytes 33215022 (33.2 MB)
        RX errors 0  dropped 1  overruns 0  frame 0
        TX packets 3026  bytes 299678 (299.6 KB)
        TX errors 0  dropped 0 overruns 0  carrier 0

In [8]:
config_threads = {}

In [9]:
try:
    s1 = slice.get_node(name=s1_name)
    s1_h1_os_iface = s1.get_interfaces()[0]
    s1_h2_os_iface = s1.get_interfaces()[1]
        
 
       
    
    file_attributes = s1.upload_file('./scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command="chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh ens7 ens8  > /tmp/script.log 2>&1'"
    # command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {s1_h1_os_iface.get_os_interface()} {s1_h2_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'"
    s1_config_thread = s1.execute_thread(command)
    config_threads[s1] = s1_config_thread
    
    print(command)


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh ens7 ens8  > /tmp/script.log 2>&1'


In [10]:
try:
    for node, thread in config_threads.items():
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        # print(f"stdout: {stdout}")
        print(f"stderr: {stderr}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 


Config thread node s1 complete
stderr: 


In [12]:
for switch_name in [s1_name]:
    switch_node = slice.get_node(name=switch_name)
    management_ip_switch = str(switch_node.get_management_ip())
    print("Swtitch Name        : {}".format(switch_node.get_name()))
    print("Management IP    : {}".format(management_ip_switch))
    
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}_commands.txt'
    print(cmd_file)
    file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))

    #stdout = execute_script(username, switch_node, f"sudo sh -c 'cat {cmd_file} | docker exec -it fabric_p4 simple_switch_CLI  > /tmp/script.log 2>&1'")
    stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))

Swtitch Name        : s1
Management IP    : 63.239.135.73
s1_commands.txt
file_attributes: -rw-rw-r--   1 1000     1000          235 01 Sep 22:37 ?
stdout: ('Obtaining JSON from switch...\nDone\nControl utility for runtime P4 table manipulation\nRuntimeCmd: Setting default action of myTunnel_exact\naction:              drop\nruntime data:        \nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:02\naction:              myTunnel_forward\nruntime data:        00:03\nEntry has been added with handle 0\nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:16\naction:              myTunnel_forward\nruntime data:        00:04\nEntry has been added with handle 1\nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:01\naction:              myTunnel_forward\nruntime data:        00:01\nEntry has been added with handle 2\nRuntimeCmd: Adding entry to exact match table myTunne

In [ ]:
h1.upload_file('scripts/send.py', 'send.py')
h1.upload_file('scripts/receive.py', 'receive.py')
h1.execute('sudo mv send.py tutorials/exercises/basic_tunnel/send_modified.py')
h1.execute('sudo mv receive.py tutorials/exercises/basic_tunnel/receive_modified.py')

In [ ]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 2 10.10.2.2 "message100"\'')

In [ ]:
h1.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')

In [ ]:
h2.upload_file('scripts/receive.py', 'receive.py')
h2.execute('sudo mv receive.py tutorials/exercises/basic_tunnel/receive_modified.py')
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 2 10.0.2.2 "message100"\'')
h2.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')